<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Vectorización


In [35]:
import numpy as np

In [36]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [37]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [38]:
documentos = [documento.split() for documento in corpus]
vocabulario =set()
for documento in documentos:
    vocabulario.update(documento)
print (vocabulario)

{'que', 'muchas', 'dia', 'gracias', 'de', 'el', 'hoy', 'es', 'martes'}


In [39]:
print (documentos)

[['que', 'dia', 'es', 'hoy'], ['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'], ['martes', 'muchas', 'gracias']]


### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [40]:
dict_vocab={palabra:i for i,palabra in enumerate(vocabulario)}
one_hot_matrix = np.zeros((len(documentos), len(vocabulario)), dtype=int)
one_hot_matrix


array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [41]:
dict_vocab

{'que': 0,
 'muchas': 1,
 'dia': 2,
 'gracias': 3,
 'de': 4,
 'el': 5,
 'hoy': 6,
 'es': 7,
 'martes': 8}

In [42]:
for n_doc,documento in enumerate(documentos):
    for palabra in documento:
        one_hot_matrix[n_doc,dict_vocab[palabra]]=1
print ("Documento 0:",one_hot_matrix[0,:])
print ("Documento 1:",one_hot_matrix[1,:])
print ("Documento 2:",one_hot_matrix[2,:])

Documento 0: [1 0 1 0 0 0 1 1 0]
Documento 1: [0 0 1 0 1 1 1 1 1]
Documento 2: [0 1 0 1 0 0 0 0 1]


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [43]:
frec_matrix= np.zeros((len(documentos), len(vocabulario)), dtype=int)
for n_doc,documento in enumerate(documentos):
    for palabra in documento:
        frec_matrix[n_doc,dict_vocab[palabra]]+=1
print ("Documento 0:",frec_matrix[0,:])
print ("Documento 1:",frec_matrix[1,:])
print ("Documento 2:",frec_matrix[2,:])

Documento 0: [1 0 1 0 0 0 1 1 0]
Documento 1: [0 0 1 0 1 1 1 1 2]
Documento 2: [0 1 0 1 0 0 0 0 1]


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [44]:
tf_idf_matrix= np.zeros((len(documentos), len(vocabulario)), dtype=float)
cant_doc=len(documentos)
idf= np.zeros (len(vocabulario),dtype=float)
for j,documento in enumerate(documentos):
    for k,palabra in enumerate(documento):
            idf[dict_vocab[palabra]]=idf[dict_vocab[palabra]]+1
print("Palabras en vocabulario:\n\t", dict_vocab.keys())
print ("Cantidad apariciones:",idf)
for i in range(len(idf)):
    idf[i]= cant_doc/idf[i]
print("Valor idf:", idf)



Palabras en vocabulario:
	 dict_keys(['que', 'muchas', 'dia', 'gracias', 'de', 'el', 'hoy', 'es', 'martes'])
Cantidad apariciones: [1. 1. 2. 1. 1. 1. 2. 2. 3.]
Valor idf: [3.  3.  1.5 3.  3.  3.  1.5 1.5 1. ]


In [45]:
for i in range(tf_idf_matrix.shape[0]):
    for j in range(tf_idf_matrix.shape[1]):
        tf_idf_matrix[i,j]=frec_matrix[i,j]*idf[j]
tf_idf_matrix

array([[3. , 0. , 1.5, 0. , 0. , 0. , 1.5, 1.5, 0. ],
       [0. , 0. , 1.5, 0. , 3. , 3. , 1.5, 1.5, 2. ],
       [0. , 3. , 0. , 3. , 0. , 0. , 0. , 0. , 1. ]])

### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [58]:
def similitud_coseno(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    return dot_product / (norm_vector1 * norm_vector2)
def ordenar_documentos_por_similitud(corpus, indice_referencia):    
    
    similitudes = []
    documento_referencia = frec_matrix[indice_referencia]
    for documento in frec_matrix:
        similitud = similitud_coseno(documento_referencia, documento)
        similitudes.append(similitud)
    
    # Obtener los índices de los documentos ordenados por similitud (excluyendo el documento de referencia)
    indices_ordenados = np.argsort(similitudes)[::-1][1:]
    
    # Obtener los documentos ordenados por similitud
    documentos_ordenados = [corpus[i] for i in indices_ordenados]
    
    return documentos_ordenados

In [64]:
indice_referencia = 2

documentos_ordenados = ordenar_documentos_por_similitud(corpus, indice_referencia)

# Imprimir los documentos ordenados por similitud
print("Doc. de referencia:", corpus[indice_referencia])
print("Documentos ordenados de mas a menos parecidos:")
for i, documento in enumerate(documentos_ordenados, start=1):
    print(f"\t Documento {i}: {documento}")

Doc. de referencia: martes muchas gracias
Documentos ordenados de mas a menos parecidos:
	 Documento 1: martes el dia de hoy es martes
	 Documento 2: que dia es hoy
